# Space

In [ ]:
import os
import logging
import pandas as pd 
from IPython.display import display, HTML
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)

import sys
sys.path.append(WORKSPACE_PATH)
from proj_space import PROJECT, TaskName, SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])

recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

# [Part 1]: develop fn_humanrec.py
 
 we want to calculate how many records each patient has under different columns. 

## [Step 1]: Create Cohort Yaml

Motivation: To create a Cohort Yaml file to preprare for next steps.

Insturction: To run the following code. 

In [ ]:
# Create a HTML link and display it
path = os.path.join(recfldtkn_config_path, 'Cohort.yaml')
full_path = os.path.join(WORKSPACE_PATH, path)
display(HTML(f'{path} <a href="{full_path}" target="_blank">Open File</a>'))

## [Step 2]: Update Cohort Yaml
Motivation: 
We want to store the Cohort information for future development. 

Aim:
To store Chort raw data path and also specify RawRootID and RoodID

Input: 
CohortInfo, RawRootID and RootID

Output:
Updated Yaml file

Instruction: 

We only need to change the following part of Cohort Yaml file, 
1. ChortInfo: This folder where we save the raw data 
2. RawRootID: HumanID in Raw Data, for WellDoc project, this is Patient, for MedStar Project, this Pid_ms
3. RootID: this si HumanID in RFT Data, in WellDoc project, this is PID



**Template**

```yaml
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  the folders where to save the data
CohortInfo:
  COHORT_NAME_XXXXXX: # <---- change this.
    cohort_label: 1
    FolderPath: $DATA_RAW$/COHORT_NAME_XXXXXX

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  where to get the raw Human data.
RawRootID: 'XXXXXXXXX' # <--- HumanID in Raw Data. 
RootID:  'XXXXXXXXX'        # <--- HumanID in RFT Data. 
```

**Update and Copy Your Yaml**


```yaml
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  the folders where to save the data
CohortInfo:
  RawData2022_CGM:
    cohort_label: 1
    FolderPath: $DATA_RAW$/RawData2022_CGM/
    
  RawData2023_CVSTDCAug:
    cohort_label: 2
    FolderPath: $DATA_RAW$/RawData2023_CVSTDCAug/
    
  RawData2023_CVSDeRxAug:
    cohort_label: 3
    FolderPath: $DATA_RAW$/RawData2023_CVSDeRxAug/

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  where to get the raw Human data.
RawRootID: 'PatientID' # <--- HumanID in Raw Data. 
RootID:  'PID'         # <--- HumanID in RFT Data. 
```

In [ ]:
display(HTML(f'{path} <a href="{full_path}" target="_blank">Open File</a>'))

## [Step 3] Cohort: Pick OneCohort

Motivation: We want to choose one cohort and test our code in this one cohort.

Aim: Specify a cohort

Input: Cohort Yaml

Output: Cohort name and Cohort label of the cohort we want to choose.

Instruction: Change ```args_information = ['--cohort_label', '1'] ```

In [ ]:
recfldtkn_config_path

In [ ]:
################### in notebook ###################  
args_information = ['--cohort_label', '1'] # < ------- change here
###################################################

In [ ]:
import argparse
from recfldtkn.configfn import load_cohort_args

cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)


my_parser = argparse.ArgumentParser(description='Process Input.')

my_parser.add_argument('--cohort_name',
                    metavar='cohort_name',
                    default = None, 
                    type=str,
                    help='the cohort_name to process')

my_parser.add_argument('--cohort_label',
                    metavar='cohort_label',
                    default = None, 
                    type=str,
                    help='the label for cohort_name to process')

args = my_parser.parse_args(args_information)

cohort_label = int(args.cohort_label)
cohort_name = [v for k, v in cohort_args['CohortInfo'].items() 
               if v['cohort_label'] == cohort_label][0]['cohort_name']

print(f'=============== {cohort_name}: {cohort_label} ======================')
print(cohort_name, cohort_label)


## [Step 4] OneCohort_Args
Motivation: We want to check one cohort information 

Aim: Display one Cohort informaiton

Input: cohort_arges

Output: NA

Instruction: Run following code.


In [ ]:
OneCohort_Args = cohort_args['CohortInfo'][cohort_name]
OneCohort_Args

In [ ]:
os.listdir(OneCohort_Args['FolderPath'])

## [Step 5] selected_source_file_suffix_list
Motivation:  we select file suffixes relates to the varied data storage formats used across different projects. For instance, in the WellDoc project, our data is stored in CSV format. By specifying file suffixes, we can accurately identify or filter files with particular extensions.

Aim: Ensures that our processing or analysis workflows interact only with the relevant types of files

Input: selected_source_file_suffix_list = ['csv']

Output: selected_file_list

Instruction: Based on the project, choose selected_source_file_suffix_list and then run following code



In [ ]:
#########################
selected_source_file_suffix_list = ['csv']
#########################


# this step will be conducted within the pipeline
selected_file_list = [file for file in os.listdir(OneCohort_Args['FolderPath']) 
                      if file.endswith(tuple(selected_source_file_suffix_list))]
print(len(selected_file_list))

In [ ]:
selected_file_list[:10]

## [Step 6] Map Raw Table file to Raw Table name

Motivation: Associating each file's data with its name as the column header can help in efficiently organizing and identifying data sourced from multiple files. 

Aim:  '05_12_2022_PatientCGMDeviceDetail.csv' to 'PatientCGMDeviceDetail'

Input: selected_file_list from last step 

Output: Raw Table name

Instruction: Run following code 



In [ ]:
import inspect
#########################
def get_tablename_from_raw_filename(file_path):
    # name = file_path.split('/')[-1].split('_df_')[0]
    tablename = file_path.split('_')[-1].split('.')[0]
    return tablename

get_tablename_from_raw_filename.fn_string = inspect.getsource(get_tablename_from_raw_filename)
#########################

In [ ]:
file_path = selected_file_list[0]

tablename = get_tablename_from_raw_filename(file_path)
print(file_path)
print(tablename)

In [ ]:
# loop the selected_file_list
l = []
for i in selected_file_list:
    tablename = get_tablename_from_raw_filename(i)
    l.append(tablename)
raw_table_name_list = l 

## [Step 7]: get_rawrootid_column

Motivation: We want to get the identifier columns as ```rawrootid```By identifying the correct identifier column name, subsequent data processing or analysis steps can use a consistent reference, regardless of the specific naming convention used in the raw data.

Aim: Get ```rawrootid```

Input: selected_file_list

Output: rawrootid for files in selected_file_list

Instruction: Change the following part according to different projects since rawrootid is different for different project group. 
```def get_rawrootid_from_raw_table_column(raw_table_columns):
    if 'PatientID' in raw_table_columns: 
        rawrootid = 'PatientID' 
    if 'UserID' in raw_table_columns:
        rawrootid = 'UserID'
    if 'PatientId' in raw_table_columns: 
        rawrootid = 'PatientId' 
    return rawrootid```

In [ ]:
##################################################
def get_rawrootid_from_raw_table_column(raw_table_columns):
    if 'PatientID' in raw_table_columns: 
        rawrootid = 'PatientID' 
    if 'UserID' in raw_table_columns:
        rawrootid = 'UserID'
    if 'PatientId' in raw_table_columns: 
        rawrootid = 'PatientId' 
    return rawrootid

get_rawrootid_from_raw_table_column.fn_string = inspect.getsource(get_rawrootid_from_raw_table_column)
##################################################


In [ ]:
# Iterate through each file in the selected file list.
for file_path in selected_file_list:
    print('\n') # Print a newline for clearer output separation.
     # Construct the full path to the file using the base folder path and the file name.
    full_path = os.path.join(OneCohort_Args['FolderPath'], file_path)
    # Check if the file is a CSV file.
    if full_path.endswith('.csv'):
        print(file_path, '<--- file_path')
        df = pd.read_csv(full_path, nrows=5)
        raw_table_columns = df.columns.tolist()
        print(raw_table_columns, '<--- raw_table_columns')
        # Identify the key identifier column from the column names list using a predefined function.
        rawrootid = get_rawrootid_from_raw_table_column(raw_table_columns)
        print(rawrootid, '<--- rawrootid')# Print the identified key identifier column name.
     # Check if the file is a Pickle file and raise an error if so, as it's not supported.
    elif full_path.endswith('.p'):
        raise ValueError(f'.p is not supported: {full_path}')
     # Raise an error for any file type that is not CSV or Pickle, indicating unsupported file type.
    else:
        raise ValueError(f'file type not supported: {full_path}')

## [Step 8]: Exclude Raw Table when we select the Patients.

Motivation: 
We only focus on the patients who have at least one records. Also, some table we don't want to consider it. For example, Patient Table. 

Aim: 
patient A: PatientTable 1, ATable 0, BTable 0, CTable 0. 

patient B: PatientTable 1, ATable 1, BTable 0, CTable 0

We don't want A because A's total record number except PatientTable is 0. 
We want B because B's total  record number except PatientTable is larger than 0.

So we put `PatientTable` into `excluded_raw_table_name`.

Instruciton: 
1. Initially, we set the list as empty
2. Based on excluded_raw_table_name is empty, we will get the df_Human (df_Human2RawRecNum) at the end of this notebook.
3. Then, we will check the df_Human, and see if there is any table that we want to exclude. If there is, we will add the table name to the excluded_raw_table_name list.

In [ ]:
# How to get the list. 

# initially, the list is empty
# based on excluded_raw_table_name is empty
# we will get the df_Human (df_Human2RawRecNum) at the end of this notebook.
# Then, we will check the df_Human, and see if there is any table that we want to exclude.
# if there is, we will add the table name to the excluded_raw_table_name list.

#########################
# excluded_raw_table_names = [] 
excluded_raw_table_names = ['UserDetail', 'Patient', 
                           'QuestionResponse', 'PatientBloodGlucoseTargets', 
                           'Rx', 'PatientObservationSummary', 
                           'PatientTargetSegment', 'TDC']
#########################

## [Step 9]: Save the above tools into the pipeline folder
Motivation: creating a more dynamic, efficient, and customizable workflow that can adapt to various data processing needs, automate repetitive tasks, and enhance the overall data analysis and handling process.

Instruction: Run the following code.


In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
iterative_variables = [selected_source_file_suffix_list, excluded_raw_table_names]
fn_variables = [get_tablename_from_raw_filename, get_rawrootid_from_raw_table_column]
pycode = convert_variables_to_pystirng(iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, 
                                       prefix = prefix)
pypath = cohort_args['pypath']
# print(pypath)
with open(pypath, 'w') as file: file.write(pycode)
# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# [Part 2] Process dfHumanRec with Pipeline


Given ../pipeline/fn_humanrec/humanrec.py  is ready. 

We have a pipeline tool: get_cohort_level_record_number_counts

This will call this humanrec.py internally, to process our records. 

In [ ]:
cohort_args['pypath']

## [Step 1]: Call Pipeline to get df_Human
Motivation: To calculate rec_num

Aim: 

Input: ```cohort_name, cohort_label, cohort_args```

Output: ```df_Human```

Instruction: 
1. Here we have a pipeline function `get_cohort_level_record_number_counts` to get the df_Human.

2. It will load the tools from '../pipeline/fn_humanrec/humanrec.py' to do the process.

If you want to know the details of the pipeline_fn, you can go to the unpacking notebook to understand it more. 

In [ ]:
from recfldtkn.pipeline_record import get_cohort_level_record_number_counts
df_Human = get_cohort_level_record_number_counts(cohort_name, cohort_label, cohort_args)
df_Human

# [Step 2] Check and Update excluded_raw_table_names
Motivation: Delect unnessary columns????

In [ ]:
df_Human

# [Step 3] Save ds_Human as HFDS

In [ ]:
import datasets
ds_HumanRec = datasets.Dataset.from_pandas(df_Human)
print(ds_HumanRec)

In [ ]:
cohort_args['RecName']

In [ ]:
print(SPACE['DATA_RFT'])

full_cohort_name = f'{cohort_label}-{cohort_name}'
print(full_cohort_name)
# ------------------------------------------------------------------------- # 
path = os.path.join(SPACE['DATA_RFT'], full_cohort_name, cohort_args['RecName'] + '_data')
print(path)
ds_HumanRec.save_to_disk(path)
print(ds_HumanRec)

# [Part 3] Select Patients with PID

In [ ]:
# from recfldtkn.loadtools import load_ds_rec_and_info
# ds_Human, _ = load_ds_rec_and_info(cohort_args['RecName'], cohort_args)
# df_Human = ds_Human.to_pandas()
# df_Human
# RootID = cohort_args['RootID']
# RawRootID = cohort_args['RawRootID']
# PID_list = [1013405, 1002538, 1022279, 1004432, 1016032, 1032308, 1031363, 1001133, 1007343, 1026067]
# print(PID_list)
# def get_patient_records_Ri(RawRootID_sample, RawRootID, cohort_args):
#     d = {}
#     cohort_config = cohort_args['CohortInfo'][cohort_name]
#     FolderPath = cohort_config['FolderPath']
#     chunk_size = 100000

#     file_list = sorted(os.listdir(FolderPath))
#     file_list = [i for i in file_list if 'csv' in i]
#     for file in file_list:
#         full_file = os.path.join(FolderPath, file)
#         li = [chunk[chunk[RawRootID] == RawRootID_sample] 
#               for chunk in pd.read_csv(full_file, chunksize=chunk_size, low_memory=False)]
#         result = pd.concat(li)
#         logger.info(f'{result.shape}: {file}')
#         if len(result) == 0: continue
#         d[file] = result
        
#     return d

# for PID_sample in PID_list:
#     print('\n======== PID_sample:', PID_sample, '========')
#     PIDInfo_dict = df_Human[df_Human[RootID] == PID_sample].iloc[0].to_dict()
#     RawRootID_sample = PIDInfo_dict[RawRootID]
#     d = get_patient_records_Ri(RawRootID_sample, RawRootID, cohort_args)
#     folder = os.path.join(SPACE['DATA_RAW'], 'patient_sample', str(PID_sample))
#     if os.path.exists(folder) == False: os.makedirs(folder)
#     for file, df in d.items():
#         df.to_csv(os.path.join(folder, file), index = False)
#         print(file, df.shape)